In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('ggplot')
import seaborn as sns
from enum import Enum
import matplotlib.colors as mcolors
import colorsys

### Make combined File

In [57]:
sdqlpy_results =  pd.read_csv("../test_results/hyper_sdqlpy_pandas_tpch_opt_all_comparison_test_results.csv") 
sdqlpy_results = sdqlpy_results[sdqlpy_results["Data Type"].isin(["SDQLpy", "Pandas"])]

sdqlpy_results['Query Name'] = sdqlpy_results['Query Name'].replace(
    {
        'Pandas Converted (Hyper DB)': 'Pandas NoOrder'
     }
)

In [58]:
pg_results =  pd.read_csv("../test_results/pg_duck_hyper_tpch_test_results.csv")
pg_results = pg_results[pg_results["Query Name"] == "Pandas Converted (Postgres)"]

In [59]:
results = pd.read_csv("../test_results/pg_duck_hyper_tpch_modif_test_results.csv") 

results = pd.concat([results, sdqlpy_results, pg_results], ignore_index=True)
results.to_csv("../test_results/pg_duck_hyper_tpch_combined_test_results.csv", index=False)
# results = results.rename(columns={"Average": "Mean"})

## Make Graphs

- Area dimensions: 84.1 wide (inches), 29 high

In [60]:
box_height = 29
box_width = 35   # Trying this for now

In [61]:
results = pd.read_csv("../test_results/pg_duck_hyper_tpch_combined_test_results.csv") 

In [62]:
print(f"Overall Results: {len(results)}")

# Filter for Correct == True and Executed == Yes
results = results[(results.Correct == True) & (results.Executed == "Yes")]
print(f"Correct Results: {len(results)}")

Overall Results: 264
Correct Results: 264


In [63]:
def g_mean(x):
    a = np.log(x)
    return np.exp(a.mean())

def pre_process(df):
    # Sort Query Name column to in-order
    def extract_number(name):
        if isinstance(name, int):
            return name
        else:
            return int(name.split(" ")[1])

    df['Query Number'] = df.apply(lambda row : extract_number(row['Query Number']), axis = 1)
    df = df.sort_values(['Query Number'])

    # Fix the runs column
    def fix_runs(runs):
        new_runs = list(str(str(runs)[1:-1]).split(","))
        for i in range(0, len(new_runs)):
            new_runs[i] = float(new_runs[i].strip())
        return new_runs

    df['Runs'] = df.apply(lambda row : fix_runs(row['Runs']), axis = 1)

    def ignore_first_gmean(runs):
        return g_mean(runs[1:])
    
    def ignore_first_mean(runs):
        return sum(runs[1:]) / len(runs[1:])

    df["Mean"] = df.apply(lambda row : ignore_first_mean(row['Runs']), axis = 1)
    df["Query Type"] = df["Query Name"]
    df = df[["Query Type", "Query Number", "Mean", "Runs"]]
    
    return df

results = pre_process(results)

In [64]:
all_named_colors = {}
all_named_colors.update(mcolors.TABLEAU_COLORS)

def handle_colour(color):
    if "tab:" in color:
        return all_named_colors[color]
    else:
        raise Exception(f"Unrecognised color format: {color}")

def scale_lightness(hex, scale_l):
    # convert rgb to hls
    rgb = mcolors.ColorConverter.to_rgb(hex)
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    return colorsys.hls_to_rgb(h, min(1, l * scale_l), s = s)

In [65]:
def make_average_graph(df, order, color, hatching, filename, in_box_height, in_box_width, x_value='Query Number', y_value='Mean'):
    assert len(order) == len(color) == len(hatching), f"Order: {len(order)} rows, Colour: {len(color)} rows and Hatching: {len(hatching)} rows"
    print(f"The DataFrame has {len(df)} rows")
    
    # plot with seaborn barplot
    sns.set(rc={'figure.figsize':(in_box_width, in_box_height)})
    # Set DPI if resolution looks bad
    sns.set(rc={'figure.dpi': 300})
    background_color = '#f1f4f5'
    # sns.set(rc={'axes.facecolor':background_color})
    plt.rcParams['patch.linewidth'] = 3.0
    plt.rcParams['patch.edgecolor'] = 'k'
    
    plt.rc('axes', labelsize=58)    # fontsize of the x and y labels
    

    ax = sns.barplot(data=df, x=x_value, y=y_value, hue='Query Type',
                     palette=color, hue_order=order)
    
    # Loop over the bars
    for bars, hatch in zip(ax.containers, hatching):
        if hatch != None:
            for bar in bars:
                bar.set_hatch(hatch)

    # Make axes weightier
    plt.setp(ax.spines.values(), linewidth=6)
    
    for spine in ax.spines.values():
        spine.set_edgecolor('k')
    
    ax.set(xlabel='', ylabel='Mean Runtime (ms)')  # , 

    # # the non-logarithmic labels 
    # ax.set_yscale("log", base=10)
    # ticks = [1, 10, 100, 1000, 10000]
    # ax.set_yticks(ticks)
    # ax.set_yticklabels(ticks)
    
    #set parameters for tick labels
    plt.tick_params(axis='x', which='major')
    plt.tick_params(axis='y', which='major', direction="out", colors="k", left=True, labelsize=55)
    
    plt.xticks(rotation=45, ha='right')
    
    # Hide x-axis tick labels
    plt.xticks([])
    
    ax.spines[['right', 'top']].set_visible(False)
    
    ax.patch.set_alpha(0.0)
    
    # The ticks
    ax.xaxis.set_tick_params(width=15)
    ax.yaxis.set_tick_params(width=15)
    
    # leg_artists = []
    # for i in range(len(order)):
    #     p = matplotlib.patches.Patch(facecolor=color[i], hatch=hatching[i])
    #     # can also explicitly declare 2nd color like this
    #     #p = matplotlib.patches.Patch(facecolor='#DCDCDC', hatch=hatch_dict[i], edgecolor='0.5')

    #     leg_artists.append(p)

    # # Add them to legend
    # ax.legend(leg_artists, order, loc='upper center', bbox_to_anchor=(0.5, 1.10),
    #           ncol=4, fancybox=True, shadow=True)

    plt.tight_layout()

    if filename != None:
        import os  
        os.makedirs(os.path.dirname(filename), exist_ok=True)  
        plt.savefig(filename, bbox_inches="tight", transparent=True)
    
    plt.show()

## Statistics

In [66]:
def make_overall_table(df):
    overall = df.groupby("Query Type").agg(
        overall_mean_runtime=("Mean", g_mean)
    )
    overall = overall.sort_values(by="overall_mean_runtime")
    overall = overall.rename(columns={"overall_mean_runtime": "Overall Mean Runtime (s)"})
    return overall

In [67]:
overall = make_overall_table(results)
overall.head(15)

,Overall Mean Runtime (s)
Query Type,
"O1,O2,O3,O4 SDQLpy",0.170093
"O1,O2,O3 SDQLpy",0.189468
"O1,O2 SDQLpy",0.202112
O1 SDQLpy,0.205774
Hyper DB,0.275704
Duck DB,0.323686
Naive SDQLpy,0.369673
Pandas Converted (Hyper DB),0.426110
Pandas NoOrder,0.524752


### High-level Optimisations Graph

1 Thread, No Hyper Threading

- Pandas By-hand
- Pandas (PostgreSQL)
- Pandas (Duck DB)
- Pandas (Hyper DB)

In [68]:
partial_results = pd.read_csv("../test_results/pg_duck_hyper_tpch_modif_test_results.csv") 
partial_results = partial_results[partial_results["Query Name"].isin(["Pandas Converted (Duck DB)", "Pandas Converted (Hyper DB)"])]

pg_results = pd.read_csv("../test_results/pg_duck_hyper_tpch_test_results.csv")
pg_results = pg_results[pg_results["Query Name"] == "Pandas Converted (Postgres)"]

hand_results = pd.read_csv("../test_results/duck_hand_tpch_test_results.csv")
hand_results = hand_results[hand_results["Query Name"] == "Pandas By-Hand"]

hl_results = pd.concat([partial_results, pg_results, hand_results], ignore_index=True)

In [69]:
hl_data = make_overall_table(pre_process(hl_results)).reset_index()

hl_order = ['Pandas By-Hand', 'Pandas Converted (Postgres)', 'Pandas Converted (Duck DB)', "Pandas Converted (Hyper DB)"]
hl_data = hl_data[hl_data["Query Type"].isin(hl_order)]

hl_data = hl_data.set_index('Query Type').loc[hl_order]
hl_data = hl_data.reset_index()

hl_data = hl_data.rename(columns={"Overall Mean Runtime (s)" : "Mean"})
# Convert to ms
hl_data['Mean'] = hl_data['Mean'].apply(lambda x : x * 1000)

In [70]:
hl_color = ["#D9D9D9"] * 4
hl_hatch = [None, None, None, None]
hl_filename = "../analysis_results/cgo/hl_graph"

make_average_graph(hl_data,
                   hl_order,
                   hl_color,
                   hl_hatch,
                   hl_filename,
                   box_height,
                   box_width,
                   "Query Type")

The DataFrame has 4 rows


### Code Generation Graph

No limit order, 1 Thread, No Hyper Threading

- Pandas (Hyper DB)
- Naive SDQLpy
- O1-O4 SDQLpy
- Hyper DB

In [71]:
sdqlpy_pd_results =  pd.read_csv("../test_results/hyper_sdqlpy_pandas_tpch_opt_all_comparison_test_results.csv") 
sdqlpy_pd_results = sdqlpy_pd_results[sdqlpy_pd_results["Data Type"].isin(["SDQLpy", "Pandas"])]
sdqlpy_pd_results['Query Name'] = sdqlpy_pd_results['Query Name'].replace(
    {
        'Pandas Converted (Hyper DB)': 'Pandas'
    }
)

hyper_results = pd.read_csv("../test_results/hyper_nlo_tpch_test_results.csv")
hyper_results = hyper_results[hyper_results["Query Name"] == "Hyper DB"]

cg_results = pd.concat([sdqlpy_pd_results, hyper_results], ignore_index=True)

In [72]:
cg_data = make_overall_table(pre_process(cg_results)).reset_index()

cg_order = ["Pandas", 'Naive SDQLpy', 'O1,O2,O3,O4 SDQLpy', 'Hyper DB']
cg_data = cg_data[cg_data["Query Type"].isin(cg_order)]

cg_data = cg_data.set_index('Query Type').loc[cg_order]
cg_data = cg_data.reset_index()

cg_data = cg_data.rename(columns={"Overall Mean Runtime (s)" : "Mean"})
# Convert to ms
cg_data['Mean'] = cg_data['Mean'].apply(lambda x : x * 1000)

In [73]:
cg_color = ["#D9D9D9"] * 4
cg_hatch = [None, None, None, None]
cg_filename = "../analysis_results/cgo/cg_graph"

make_average_graph(cg_data,
                   cg_order,
                   cg_color,
                   cg_hatch,
                   cg_filename,
                   box_height,
                   box_width,
                   "Query Type")

The DataFrame has 4 rows


### Database Compare Graph

In [74]:
log_graph = overall.reset_index()

db_comp_order = ['Pandas Converted (Postgres)', 'Pandas Converted (Duck DB)', "Pandas Converted (Hyper DB)"]
db_comp_data = log_graph[log_graph["Query Type"].isin(db_comp_order)]

db_comp_data = db_comp_data.set_index('Query Type').loc[db_comp_order]
db_comp_data = db_comp_data.reset_index()

db_comp_data = db_comp_data.rename(columns={"Overall Mean Runtime (s)" : "Mean"})
db_comp_data['Mean'] = db_comp_data['Mean'].apply(lambda x : x * 1000)

In [75]:
# Pandas (PostgreSQL), Pandas (Duck DB), HyperDB, Pandas (Hyper)
db_comp_color = ["#88CCEE", "#DDCC77", "#CC6677"]
db_comp_hatch = [None, None, None]
db_comp_filename = "../analysis_results/cgo/database_graphs"

make_average_graph(db_comp_data,
                   db_comp_order,
                   db_comp_color,
                   db_comp_hatch,
                   db_comp_filename,
                   box_height,
                   box_width,
                   "Query Type")

The DataFrame has 3 rows


### SDQLpy Compare Graph

In [76]:
log_graph = overall.reset_index()

log_graph['Query Type'] = log_graph['Query Type'].replace(
    {
        'Pandas NoOrder': 'Pandas',
        'O1,O2 SDQLpy': 'O1-O2 SDQLpy',
        'O1,O2,O3 SDQLpy': 'O1-O3 SDQLpy',
        'O1,O2,O3,O4 SDQLpy': 'O1-O4 SDQLpy',
     }
)

sdqlpy_comp_order = ['Pandas', 'Naive SDQLpy', 'O1-O4 SDQLpy', 'Hyper DB']
sdqlpy_comp_data = log_graph[log_graph["Query Type"].isin(sdqlpy_comp_order)]

sdqlpy_comp_data = sdqlpy_comp_data.set_index('Query Type').loc[sdqlpy_comp_order]
sdqlpy_comp_data = sdqlpy_comp_data.reset_index()

sdqlpy_comp_data = sdqlpy_comp_data.rename(columns={"Overall Mean Runtime (s)" : "Mean"})
sdqlpy_comp_data['Mean'] = sdqlpy_comp_data['Mean'].apply(lambda x : x * 1000)

In [77]:
# PANDAS (Hyper), Naive SDQLpy, O1-O4, HyperDB
sdqlpy_comp_color = ['#CC6677', '#117733', '#44AA99', '#882255']
sdqlpy_comp_hatch = [None, None, None, None]
sdqlpy_comp_filename = "../analysis_results/cgo/sdqlpy_graphs"

make_average_graph(sdqlpy_comp_data,
                   sdqlpy_comp_order,
                   sdqlpy_comp_color,
                   sdqlpy_comp_hatch,
                   sdqlpy_comp_filename,
                   box_height,
                   box_width,
                   "Query Type")

The DataFrame has 4 rows


### Speedups

In [78]:
# Should be: (OLD, NEW)
relevant_pairs = [
    ("Postgres", "Pandas Converted (Postgres)"),
    ("Duck DB", "Pandas Converted (Duck DB)"),
    ("Hyper DB", "Pandas NoOrder")
]

def get_from_df_by_index(index_value):
    return float(overall.loc[[index_value]]["Overall Mean Runtime (s)"].iloc[0])

def faster_times_in_time(old_time, new_time):
    if old_time > new_time:
        return True, round(old_time/new_time, 2)
    else: 
        return False, round(new_time/old_time, 2)

for old, new in relevant_pairs:
    old_value = get_from_df_by_index(old)
    new_value = get_from_df_by_index(new)
    faster, value = faster_times_in_time(old_value, new_value)
    if faster:
        print(f"{new} was {value}x faster than {old}")
    else:
        print(f"{new} was {value}x slower than {old}")

Pandas Converted (Postgres) was 3.47x faster than Postgres
Pandas Converted (Duck DB) was 2.0x slower than Duck DB
Pandas NoOrder was 1.9x slower than Hyper DB


### Pandas Graph

In [79]:
log_graph = overall.reset_index()

# 'Pandas Converted (Postgres)',
# 'Pandas Converted (Duck DB)', 

all_6_order = ["Postgres", 'Duck DB', "Hyper DB",  "Pandas Converted (Hyper DB)"]
all_6_data = log_graph[log_graph["Query Type"].isin(all_6_order)]

all_6_data = all_6_data.set_index('Query Type').loc[all_6_order]
all_6_data = all_6_data.reset_index()

all_6_data = all_6_data.rename(columns={"Overall Mean Runtime (s)" : "Mean"})
all_6_data['Mean'] = all_6_data['Mean'].apply(lambda x : x * 1000)

In [80]:
# PostgreSQL, Duck DB, HyperDB, Pandas (Hyper)
all_6_color = ["#9E00A4", "#E4DE2F", "#d62e4e", "#ff4000"]
all_6_hatch = [None, None, None, None]
all_6_filename = "../analysis_results/cgo/pandas_graphs"

make_average_graph(all_6_data,
                   all_6_order,
                   all_6_color,
                   all_6_hatch,
                   all_6_filename,
                   "Query Type")

TypeError: make_average_graph() missing 1 required positional argument: 'in_box_width'